## Deliverable 2. Create a Customer Travel Destinations Map.

In [7]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [8]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.29,-156.79,42.8,81,90,10.29,overcast clouds
1,1,Ushuaia,AR,-54.80,-68.30,39.2,93,90,24.16,overcast clouds
2,2,Castro,BR,-24.79,-50.01,52.0,96,100,2.15,light rain
3,3,Avarua,CK,-21.21,-159.78,75.2,78,100,20.80,overcast clouds
4,4,Mahebourg,MU,-20.41,57.70,68.0,82,75,11.41,shower rain


In [9]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [10]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Avarua,CK,-21.21,-159.78,75.20,78,100,20.80,overcast clouds
16,16,Kapaa,US,22.08,-159.32,84.20,66,1,14.99,light rain
19,19,Port Blair,IN,11.67,92.75,82.17,79,100,14.25,overcast clouds
22,22,Guerrero Negro,MX,27.98,-114.06,84.02,50,0,17.90,light rain
28,28,Itarema,BR,-2.92,-39.92,77.00,77,35,13.87,scattered clouds
29,29,Port Moresby,PG,-9.44,147.18,80.60,69,75,5.82,smoke
32,32,Panguna,PG,-6.32,155.48,82.38,62,21,2.24,few clouds
33,33,Lynn Haven,US,30.25,-85.65,84.99,78,20,4.70,light rain
35,35,Belize,BZ,17.25,-88.75,82.00,77,82,3.00,broken clouds
36,36,Dingle,PH,11.00,122.67,75.76,92,90,5.10,overcast clouds


In [30]:
# 4a. Determine if there are any empty rows.
nan_value = float("NaN")
# Formula taken from Stack Overflow: https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
preferred_cities_df = preferred_cities_df.applymap(lambda x: nan_value if isinstance(x, str) 
                                                   and (not x or x.isspace()) else x)
preferred_cities_df.isna().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [13]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_preferred_cities_df = preferred_cities_df.dropna()

In [14]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,
16,Kapaa,US,84.20,light rain,22.08,-159.32,
19,Port Blair,IN,82.17,overcast clouds,11.67,92.75,
22,Guerrero Negro,MX,84.02,light rain,27.98,-114.06,
28,Itarema,BR,77.00,scattered clouds,-2.92,-39.92,
29,Port Moresby,PG,80.60,smoke,-9.44,147.18,
32,Panguna,PG,82.38,few clouds,-6.32,155.48,
33,Lynn Haven,US,84.99,light rain,30.25,-85.65,
35,Belize,BZ,82.00,broken clouds,17.25,-88.75,
36,Dingle,PH,75.76,overcast clouds,11.00,122.67,


In [15]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add latitude and longitude to location key for the params dictionary. 
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except: 
        pass
    
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,Paradise Inn
16,Kapaa,US,84.20,light rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
19,Port Blair,IN,82.17,overcast clouds,11.67,92.75,Fortune Resort Bay Island
22,Guerrero Negro,MX,84.02,light rain,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
28,Itarema,BR,77.00,scattered clouds,-2.92,-39.92,Windvillage
29,Port Moresby,PG,80.60,smoke,-9.44,147.18,Lamana Hotel
32,Panguna,PG,82.38,few clouds,-6.32,155.48,
33,Lynn Haven,US,84.99,light rain,30.25,-85.65,Wingate by Wyndham Panama City Area Lynn Haven
35,Belize,BZ,82.00,broken clouds,17.25,-88.75,Villa San Juan
36,Dingle,PH,75.76,overcast clouds,11.00,122.67,Camp Pasica


In [28]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
# Formula taken from Stack Overflow: https://stackoverflow.com/questions/13445241/replacing-blank-values-white-space-with-nan-in-pandas
hotel_df = hotel_df.applymap(lambda x: nan_value if isinstance(x, str) and (not x or x.isspace()) else x)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Avarua,CK,75.20,overcast clouds,-21.21,-159.78,Paradise Inn
16,Kapaa,US,84.20,light rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
19,Port Blair,IN,82.17,overcast clouds,11.67,92.75,Fortune Resort Bay Island
22,Guerrero Negro,MX,84.02,light rain,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
28,Itarema,BR,77.00,scattered clouds,-2.92,-39.92,Windvillage
29,Port Moresby,PG,80.60,smoke,-9.44,147.18,Lamana Hotel
33,Lynn Haven,US,84.99,light rain,30.25,-85.65,Wingate by Wyndham Panama City Area Lynn Haven
35,Belize,BZ,82.00,broken clouds,17.25,-88.75,Villa San Juan
36,Dingle,PH,75.76,overcast clouds,11.00,122.67,Camp Pasica
37,The Valley,AI,84.20,few clouds,18.22,-63.06,CeBlue Villas & Beach Resort


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [41]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))